In [126]:
## ============================================================
# PROJET FINAL TD8 / TD9 / TD10
# Interface complète sur corpus IA
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

from Corpus import Corpus
from SearchEngine import SearchEngine


# ============================================================
# 1) Chargement du corpus
# ============================================================

corpus = Corpus.load("data/corpus.csv")
engine = SearchEngine(corpus)


rows = []
for doc_id, doc in corpus.id2doc.items():
    rows.append({
        "id": doc_id,
        "auteur": getattr(doc, "auteur", "inconnu"),
        "source": str(getattr(doc, "source", "autre")),
        "titre": getattr(doc, "titre", ""),
        "texte": getattr(doc, "texte", ""),
        "date": getattr(doc, "date", None)
    })

df = pd.DataFrame(rows)

# Dates propres
df["date_dt"] = pd.to_datetime(df["date"], errors="coerce")


# ============================================================
# 2) Nettoyage des sources
# ============================================================

def normalize_source(s):
    s = s.lower()
    if "reddit" in s:
        return "Reddit"
    if "arxiv" in s:
        return "Arxiv"
    return "Autre"

df["source"] = df["source"].apply(normalize_source)
sources = sorted(df["source"].unique())


header = widgets.HTML(
    f"""
    <h2>Exploration d’un corpus – Intelligence Artificielle</h2>
    <p>
        Documents : <b>{len(df)}</b> |
        Auteurs : <b>{df["auteur"].nunique()}</b> |
        Sources : <b>{", ".join(sources)}</b>
    </p>
    """
)




# ============================================================
# OUTILS TEXTE
# ============================================================

def tokenize(txt):
    return [w.lower() for w in txt.split() if w.isalpha()]


# ============================================================
# ONGLET 0 — Statistiques globales
# ============================================================

out_stats = widgets.Output()

with out_stats:
    display(
        pd.DataFrame({
            "Documents": df.groupby("source").size(),
            "Auteurs": df.groupby("source")["auteur"].nunique()
        })
    )

tab_stats = widgets.VBox([
    widgets.HTML("<h3>Statistiques globales</h3>"),
    out_stats
])


# ============================================================
# ONGLET 1 — Recherche
# ============================================================

query = widgets.Text(description="Requête")
topk = widgets.IntSlider(value=10, min=1, max=30, description="Top")

src_filter = widgets.Dropdown(options=["(tous)"] + sources, description="Source")
auth_filter = widgets.Dropdown(options=["(tous)"], description="Auteur")

btn_search = widgets.Button(description="Rechercher", button_style="primary")
out_search = widgets.Output()

def update_authors(_=None):
    if src_filter.value == "(tous)":
        sub = df
    else:
        sub = df[df["source"] == src_filter.value]
    auth_filter.options = ["(tous)"] + sorted(sub["auteur"].unique())

src_filter.observe(update_authors, names="value")
update_authors()
def run_search(_=None):
    with out_search:
        out_search.clear_output()

        q = query.value.strip()
        if not q:
            print("Veuillez entrer une requête.")
            return

        # 1) Sécuriser l'appel au moteur
        try:
            res = engine.search(q, int(topk.value))
        except Exception:
            print("Requête invalide ou erreur dans le moteur de recherche.")
            return

        # 2) Si le moteur renvoie None / pas un DataFrame / vide
        if res is None or not isinstance(res, pd.DataFrame) or res.empty:
            print("Aucun résultat.")
            return

        # 3) Si la colonne id n'existe pas, on n'essaie pas de merge
        if "id" not in res.columns:
            print("Aucun résultat.")
            return

        # 4) Merge sécurisé
        try:
            res = res.merge(df[["id", "auteur", "source"]], on="id", how="left")
        except Exception:
            print("Aucun résultat.")
            return

        # Nettoyage colonnes (inchangé)
        res = res.drop(columns=[c for c in res.columns if c.endswith("_x")], errors="ignore")
        res = res.rename(columns={"auteur_y": "auteur"})

        # Filtres (inchangés)
        if src_filter.value != "(tous)":
            res = res[res["source"] == src_filter.value]

        if auth_filter.value != "(tous)":
            res = res[res["auteur"] == auth_filter.value]

        if res.empty:
            print("Aucun résultat.")
            return

        # Affichage (inchangé)
        display(res[["score", "id", "titre", "auteur", "source"]])

        plt.figure(figsize=(6,3))
        plt.bar(range(len(res)), res["score"])
        plt.xlabel("Résultats")
        plt.ylabel("Score")
        plt.title("Scores des documents")
        plt.show()


btn_search.on_click(run_search)

tab_search = widgets.VBox([
    widgets.HTML("<h3>Recherche par mots-clés</h3>"),
    query, topk, src_filter, auth_filter,
    btn_search, out_search
])


# ============================================================
# ONGLET 2 — Comparaison Reddit / Arxiv 
# ============================================================

srcA = widgets.Dropdown(options=sources, description="Source A")
srcB = widgets.Dropdown(options=sources, description="Source B")
top_words = widgets.IntSlider(value=20, min=5, max=50, step=5, description="Top mots")

btn_cmp = widgets.Button(description="Comparer", button_style="primary")
out_cmp = widgets.Output()

def comparer_sources(dfA, dfB, top):
    fA, fB = Counter(), Counter()
    for t in dfA["texte"]:
        fA.update(tokenize(t))
    for t in dfB["texte"]:
        fB.update(tokenize(t))

    mots = set(fA) | set(fB)
    rows = []

    for m in mots:
        rows.append({
            "mot": m,
            f"{srcA.value}_freq": fA.get(m, 0),
            f"{srcB.value}_freq": fB.get(m, 0),
            "diff": fA.get(m, 0) - fB.get(m, 0)
        })

    return pd.DataFrame(rows).sort_values("diff", ascending=False).head(top)

#calcul TF/DF global 
def compute_global_tf_df():
    tf = Counter()
    dfreq = Counter()
    for t in df["texte"]:
        toks = tokenize(t)
        tf.update(toks)
        dfreq.update(set(toks))
    rows = [{"mot": m, "TF": tf[m], "DF": dfreq[m]} for m in tf.keys()]
    return pd.DataFrame(rows).sort_values("TF", ascending=False)

# Pré-calcul une seule fois 
GLOBAL_TF_DF = compute_global_tf_df()

def run_cmp(_=None):
    with out_cmp:
        out_cmp.clear_output()

        if srcA.value == srcB.value:
            print("Choisissez deux sources différentes.")
            return

        # Comparaison existante (inchangée)
        display(
            comparer_sources(
                df[df["source"] == srcA.value],
                df[df["source"] == srcB.value],
                int(top_words.value)
            )
        )

        # Ajout TF/DF global 
        display(widgets.HTML("<b>Top mots globaux (TF / DF)</b>"))
        display(GLOBAL_TF_DF.head(15))

btn_cmp.on_click(run_cmp)

tab_cmp = widgets.VBox([
    widgets.HTML("<h3>Comparaison lexicale Reddit / Arxiv</h3>"),
    srcA, srcB, top_words,
    btn_cmp, out_cmp
])


# ============================================================
# ONGLET 3 — Évolution temporelle
# ============================================================

mot_time = widgets.Text(
    description="Mot",
    placeholder="ex : intelligence",
    style={"description_width": "initial"}
)

date_start = widgets.DatePicker(
    description="Début",
    style={"description_width": "initial"}
)

date_end = widgets.DatePicker(
    description="Fin",
    style={"description_width": "initial"}
)

src_time = widgets.Dropdown(
    options=["(tous)"] + sources,
    description="Source",
    style={"description_width": "initial"}
)

btn_time = widgets.Button(
    description="Afficher",
    button_style="primary"
)

out_time = widgets.Output()


def run_time(_=None):
    with out_time:
        out_time.clear_output()

        #sécurité entrée utilisateur
        mot = mot_time.value.lower().strip()
        if mot == "":
            print("Veuillez entrer un mot.")
            return

        #copie du dataframe
        dfx = df.copy()

        # gestion des dates
        dfx["date_dt"] = pd.to_datetime(dfx["date"], errors="coerce", utc=True)
        dfx["date_dt"] = dfx["date_dt"].dt.tz_convert(None)

        dfx = dfx.dropna(subset=["date_dt"])

        # filtres
        if src_time.value != "(tous)":
            dfx = dfx[dfx["source"] == src_time.value]

        if date_start.value:
            dfx = dfx[dfx["date_dt"] >= pd.Timestamp(date_start.value)]

        if date_end.value:
            dfx = dfx[dfx["date_dt"] <= pd.Timestamp(date_end.value)]

        if dfx.empty:
            print("Aucune donnée pour ces filtres.")
            return

        # -comptage du mot
        dfx["count"] = dfx["texte"].apply(lambda t: tokenize(t).count(mot))
        dfx["year"] = dfx["date_dt"].dt.year

        df_time = (
            dfx.groupby("year")["count"]
            .sum()
            .reset_index()
            .rename(columns={"year": "Année", "count": "Fréquence"})
        )

        if df_time["Fréquence"].sum() == 0:
            print(f"Le mot '{mot}' n'apparaît pas sur cette période.")
            return

        display(df_time)

        plt.figure(figsize=(7,4))
        plt.bar(df_time["Année"], df_time["Fréquence"])
        plt.xlabel("Année")
        plt.ylabel("Nombre d'occurrences")
        plt.title(f"Évolution du mot '{mot}'")
        plt.xticks(rotation=45)
        plt.show()

        total = int(df_time["Fréquence"].sum())
        pic = df_time.loc[df_time["Fréquence"].idxmax()]

        print(
            f"Analyse automatique :\n"
            f"Le mot '{mot}' apparaît {total} fois sur la période sélectionnée.\n"
            f"Le pic d'utilisation est observé en {int(pic['Année'])} "
            f"avec {int(pic['Fréquence'])} occurrences."
        )


btn_time.on_click(run_time)

tab_time = widgets.VBox([
    widgets.HTML("<h3>Évolution temporelle d’un mot</h3>"),
    mot_time,
    widgets.HBox([date_start, date_end]),
    src_time,
    btn_time,
    out_time
])


# ============================================================
# ONGLET 4 — Exploration par auteur
# ============================================================

auth_select = widgets.Dropdown(
    options=sorted(df["auteur"].unique()),
    description="Auteur"
)

out_auth = widgets.Output()

def show_author(_=None):
    with out_auth:
        out_auth.clear_output()
        sub = df[df["auteur"] == auth_select.value]
        print(f"Documents : {len(sub)}")
        display(sub[["titre", "source", "date"]].head(10))

auth_select.observe(show_author, names="value")
show_author()

tab_auth = widgets.VBox([
    widgets.HTML("<h3>Exploration par auteur</h3>"),
    auth_select, out_auth
])


# ============================================================
# AFFICHAGE FINAL
# ============================================================

tabs = widgets.Tab(children=[
    tab_stats,
    tab_search,
    tab_cmp,
    tab_time,
    tab_auth
])

tabs.set_title(0, "Stats")
tabs.set_title(1, "Recherche")
tabs.set_title(2, "Comparaison")
tabs.set_title(3, "Temps")
tabs.set_title(4, "Auteurs")

display(widgets.VBox([header, tabs]))




📂 Corpus chargé : 15 documents, 21 auteurs
